In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.models.load_model('model/model.h5')
# # model.summary()
# # model.get_weights()

img_path = "img_predict/verifyImg.png"
max_length = 6
img_width = 150
img_height = 50

prediction_model = keras.models.Model(
    model.get_layer(name="image").input, model.get_layer(name="dense2").output
)

vocab = ['2', '3', '4', '5', '6', '7', '8', 'A', 'B', 'C', 'D', 'F', 'J', 'N', 'P', 'W', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'm', 'n', 'o', 'p', 'r', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    
num_to_char = layers.StringLookup(
    vocabulary=vocab, mask_token=None, invert=True
)

def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_length
    ]
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text

def classify_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.transpose(img, perm=[1, 0, 2])
    img = tf.expand_dims(img, axis=0)
    preds = prediction_model.predict(img)
    pred_text = decode_batch_predictions(preds)
    return pred_text[0]

predict = classify_image(img_path)
print(predict)

1/1 [==============================] - 2s 2s/step
[UNK][UNK][UNK][UNK][UNK][UNK]
